In [4]:
import pandas as pd
import numpy as np

In [5]:
canada_postal_codes = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', attrs={'class': 'sortable'})[0]
canada_postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
canada_postal_codes.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
canada_postal_codes = canada_postal_codes[canada_postal_codes['Borough']!='Not assigned']
canada_postal_codes.reset_index(drop=True, inplace=True)
canada_postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [7]:
grouped = canada_postal_codes.groupby('PostalCode')
Neighborhood = grouped.apply(lambda d: ','.join(d['Neighborhood'])).to_frame()
Borough = grouped.apply(lambda d: pd.unique(d['Borough'])[0]).to_frame()
canada_postal_codes_cleaned = Borough.merge(Neighborhood, on='PostalCode')
canada_postal_codes_cleaned.rename(columns={'0_x': 'Borough', '0_y': 'Neighborhood'}, inplace=True)
canada_postal_codes_cleaned.reset_index(inplace=True)
canada_postal_codes_cleaned.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Locate rows with Neighborhood 'Not assigned' and set to the borough name.

In [8]:
canada_postal_codes_cleaned[canada_postal_codes_cleaned['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [9]:
canada_postal_codes_cleaned.at[85, 'Neighborhood'] = canada_postal_codes_cleaned.at[85, 'Borough']

In [10]:
canada_postal_codes_cleaned.loc[85]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 85, dtype: object

In [8]:
canada_postal_codes_cleaned.shape

(103, 3)

In [19]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# !conda install beautifulsoup4 --yes
from bs4 import BeautifulSoup
import requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [39]:
from geopy.geocoders import Nominatim # geocoder.google deny the request, using geopy
geolocator = Nominatim(user_agent="ny_explorer")

def getLatLng(postal_code):
    location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = location and [location.latitude, location.longitude] or [None, None]
    return lat_lng_coords


In [ ]:

latlng = canada_postal_codes_cleaned.apply(lambda x: pd.Series(getLatLng(x.at['PostalCode']), index=['Latitude', 'Longitude']), axis=1)
latlng

In [24]:
lat_lng_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
lat_lng_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
toronto_df = canada_postal_codes_cleaned.merge(lat_lng_df, on='PostalCode')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
